# Create Policy Tags from DLP Scan Results

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=

In [ ]:
%env CLOUDSDK_PYTHON=python

In [ ]:
from google.cloud import bigquery

## Get DLP results from BigQuery

See tutorial from https://cloud.google.com/dlp/docs/querying-findings and https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries

Here is the schema of the table.

![](schema.png)

Here is the InfoTypes that were scanned.

![](dlp_scan.png)

The detailed results of DLP must be stored on BigQuery (`dlp_table_id`). First, we query the results of DLP from BigQuery. Consider only the results with POSSIBLE, LIKELY, or VERY LIKELY outcomes and have at least `min_rec` records per column per PII type. Obtain the column name that contains PII from `location.content_locations.record_location.field_id.name` column, and the type of PII from `info_type.name` column.

In [ ]:
project_id = ''
location = ''
dataset_id = ''
dlp_table_id = ''
table_id = ''
min_rec = 5
policy_tag_tax_id = ''

In [ ]:
client = bigquery.Client()

In [ ]:
query_job = client.query(
    f"""
    SELECT
      table_counts.field_name,
      STRING_AGG( CONCAT(" ",table_counts.name," [count: ",CAST(table_counts.count_total AS String),"]")
      ORDER BY
        table_counts.count_total DESC) AS infoTypes
    FROM (
      SELECT
        locations.record_location.field_id.name AS field_name,
        info_type.name,
        COUNT(*) AS count_total
      FROM
        `{project_id}.{dataset_id}.{dlp_table_id}`,
        UNNEST(location.content_locations) AS locations
      WHERE
        (likelihood = 'LIKELY'
          OR likelihood = 'VERY_LIKELY'
          OR likelihood = 'POSSIBLE')
      GROUP BY
        locations.record_location.field_id.name,
        info_type.name
      HAVING
        count_total > {min_rec} ) AS table_counts
    GROUP BY
      table_counts.field_name
    ORDER BY
      table_counts.field_name
    """
)

results = query_job.result()  # Waits for job to complete.

In [ ]:
col = []
val = []
for row in results:
    col.append(row[0])
    val.append(row[1])
    print(row)

Get columns to tag

In [ ]:
tags = {k:v.split(',')[0].strip().split(' ')[0] for k, v in zip(col, val)}
tags

## Assign Policy Tags to BigQuery Columns

First, create policy tag taxonomy using Google cloud console, which contains one tag per PII type. Then, run the following lines.

In [ ]:
from google.cloud import datacatalog_v1beta1

In [ ]:
client = datacatalog_v1beta1.PolicyTagManagerClient()

Get the policy tag taxonomy.

In [ ]:
name = client.taxonomy_path(project_id, location, policy_tag_tax_id)

In [ ]:
policy_tags = {element.display_name:element.name 
               for element in client.list_policy_tags(parent=name)}
policy_tags

Get the content (table) that we want to tag.

In [ ]:
table = f'{project_id}:{dataset_id}.{table_id}'
table

Get the original schma of the table.

In [ ]:
!bq show --schema --format=prettyjson $table > schema.json

In [ ]:
import json

In [ ]:
with open("schema.json", "r") as f:
    schema_dict = json.loads(f.read())

print(schema_dict)

Add policy tag to the schema

In [ ]:
for field in schema_dict:
    if field['name'] in tags.keys():
        policy_tag = policy_tags[tags[field['name']]]
        field['policyTags'] = {'names':[ policy_tag ]}

In [ ]:
schema_dict

Update the schema

In [ ]:
with open('new_schema.json', 'w') as outfile:
    json.dump(schema_dict, outfile)

In [ ]:
!bq update $table new_schema.json

Here is the new schema of the table.

![](new_schema.PNG)